<a href="https://colab.research.google.com/github/Sammers45/Brain-Tumor-Detection/blob/master/CNNOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create CNN Model and Optimize it using Keras Tuner


In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
print(tf.__version__)

2.17.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [11]:
train_images.shape

(60000, 28, 28, 1)

In [14]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5]),
        activation = 'relu',
        input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size = hp.Choice('conv_2_kernel', values=[3,5]),
        activation = 'relu'),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='MNIST_Fashion')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
tuner_search.search(train_images, train_labels, epochs=2, validation_split=0.1)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.859333336353302

Best val_accuracy So Far: 0.9006666541099548
Total elapsed time: 00h 02m 11s


In [20]:
model = tuner_search.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 80)          │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 48)          │          34,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 23232)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 96)                  │       2,230,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             970 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,268,026 (8.65 MB)

 Trainable params: 2,268,026 (8.65 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.1, initial_epoch=2)

Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9223 - loss: 0.2117 - val_accuracy: 0.9065 - val_loss: 0.2509
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9412 - loss: 0.1574 - val_accuracy: 0.9065 - val_loss: 0.2701
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9547 - loss: 0.1196 - val_accuracy: 0.9113 - val_loss: 0.2696
